In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Define a list of models or algorithms to compare
models = [
    ('Random Forest', RandomForestClassifier()),
    ('KNN', KNeighborsClassifier()),
    ('SVM', SVC()),
    ('Perceptron', MLPClassifier(hidden_layer_sizes=(100, ))),
]

# Define a dictionary of hyperparameter grids for each model
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'Perceptron': {
        'alpha': [0.0001, 0.001, 0.01],
        'activation': ['logistic', 'relu'],
    },
}

# Outer loop for cross-validation
outer_cv = KFold(n_splits=5, shuffle=True)

# Initialize a dictionary to store the evaluation scores for each model
model_scores = {}

# Outer loop for cross-validation
for train_index, test_index in outer_cv.split(X):

    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Inner loop for model selection and hyperparameter tuning
    inner_cv = KFold(n_splits=5, shuffle=True)

    for model_name, model in models:

        # Define the hyperparameter grid for the current model
        param_grid = param_grids[model_name]

        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv)
        grid_search.fit(X_train, y_train)

        # Get the best model from the inner loop
        best_model = grid_search.best_estimator_

        # Evaluate the best model on the testing set
        y_pred = best_model.predict(X_test)
        score = metrics.accuracy_score(y_test, y_pred)

        # Store the evaluation score for the current model
        if model_name in model_scores:
            model_scores[model_name].append(score)
        else:
            model_scores[model_name] = [score]

# Calculate the mean and standard deviation of the evaluation scores for each model
mean_scores = {model_name: np.mean(scores) for model_name, scores in model_scores.items()}
std_scores = {model_name: np.std(scores) for model_name, scores in model_scores.items()}


# Print the results for each model
for model_name, mean_score in mean_scores.items():
    std_score = std_scores[model_name]
    print(f"Model: {model_name}")
    print("Mean Accuracy:", mean_score)
    print("Standard Deviation:", std_score)
    print()